In [ ]:
import torch
from huggingface_hub import hf_hub_download
from safetensors import
from anydoor_refiners.model import AnyDoor

In [ ]:
anydoor = AnyDoor(
    device="cuda" if torch.cuda.is_available() else "cpu",
    dtype=torch.float16,
)

In [ ]:
anydoor.unet.load_from_safetensors("../ckpt/unet_refiners.safetensors")
anydoor.control_model.load_from_safetensors("../ckpt/controlnet.safetensors")
anydoor.object_encoder.load_from_safetensors("../ckpt/encoder.safetensors")
anydoor.lda.load_from_safetensors(
    hf_hub_download(
        repo_id="refiners/sd15.autoencoder",
        filename="model.safetensors",
    )
)

In [ ]:
# hyperparameters
seed = 42
uncod_scale = 5.0
num_inference_steps = 50
anydoor.set_inference_steps(num_inference_steps, first_step=0)

In [ ]:
inserted_object_img : torch.Tensor = ...
background_control_img : torch.Tensor = ...

object, control = preprocessing(inserted_object_img,background_control_img)

In [ ]:
from refiners.fluxion.utils import manual_seed, no_grad
from tqdm import tqdm

with no_grad():  
    manual_seed(seed)
    conditionning = anydoor.compute_conditionning(object,control)
    x = anydoor.init_latents((64, 64))

    # diffusion denoising process
    for step in tqdm(anydoor.steps):
        x = anydoor.forward(
            x,
            step=step,
            conditionning=conditionning,
            condition_scale=uncod_scale
        )
    predicted_image = anydoor.lda.latents_to_image(x)

predicted_image.save("vanilla_sdxl.png")